In [1]:
import caffe

In [106]:
from caffe import layers as L, params as P, to_proto

In [3]:
from caffe.proto import caffe_pb2

In [4]:
# data, label = L.Data(source="./train.txt", backend=P.Data.LMDB, batch_size=3, ntop=2,
#         transform_param=dict(crop_size=227, mean_value=[104, 117, 123], mirror=True))

In [100]:
def conv_relu(bottom, ks, nout, stride=1, pad=0, group=1):
    conv = L.Convolution(bottom, kernel_size=ks, stride=stride,
                                num_output=nout, pad=pad, group=group)
    return L.ReLU(conv, in_place=True)

def fc_relu(bottom, nout):
    fc = L.InnerProduct(bottom, num_output=nout)
    return fc, L.ReLU(fc, in_place=True)

def max_pool(bottom, ks=2, stride=2):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=ks, stride=stride)

In [58]:
def deconv_relu(bottom, ks, nout, stride=1, pad=0, group=1):
    convolution_param = dict(num_output=nout, kernel_size=ks, stride=stride,pad=pad, group=group,
            bias_term=False)
    deconv = L.Deconvolution(bottom,convolution_param=convolution_param)
    return L.ReLU(deconv, in_place=True)
    
    
    

In [121]:
batch_size=1
data =L.NetCDFData(source="train.txt", variable_data=['PRECT',
                                                     'PS',
                                                     'PSL',
                                                     'QREFHT',
                                                     'T200',
                                                     'T500',
                                                     'TMQ',
                                                     'TREFHT',
                                                     'TS',
                                                     'U850',
                                                     'UBOT',
                                                     'V850',
                                                     'VBOT',
                                                     'Z1000',
                                                     'Z200', 'ZBOT'],
                   variable_label = ["teca_mask"], first_dim_is_batched=True,
                 batch_size=batch_size)


data = L.DummyData(shape={'dim':[batch_size,16,768,1152]})
label = L.DummyData(shape={'dim':[batch_size,6,12,18]})


nfilters = 32

conv = data

num_layers = 6

for i in range(num_layers):
    nfilters *= 2
    conv = conv_relu(conv, ks=5, nout=nfilters, pad=2, stride=1)
    conv = max_pool(conv)
    
conv = L.Convolution(conv,kernel_size=1,num_output=6, stride=1 )

loss = L.EuclideanLoss(conv, label)

prototxt_str = str(to_proto(loss))

# #man oh man this is hacky
# #add in second top for netcdf data layer
# str_of_int = 'top: "NetCDFData1"\n '

# insert_ind = prototxt_str.index(str_of_int) + len(str_of_int)
# insert_str = ' top: "NetCDFData2"\n ' 
# prototxt_str = prototxt_str[:insert_ind] + insert_str + prototxt_str[insert_ind:]

# prototxt_str = prototxt_str.replace("NetCDFData1", "data")
# prototxt_str =prototxt_str.replace("NetCDFData2", "label")

# #print prototxt_str
with open('train.prototxt', 'w') as f:
        f.write(prototxt_str)

In [122]:
print prototxt_str

layer {
  name: "DummyData1"
  type: "DummyData"
  top: "DummyData1"
  dummy_data_param {
    shape {
      dim: 1
      dim: 16
      dim: 768
      dim: 1152
    }
  }
}
layer {
  name: "Convolution1"
  type: "Convolution"
  bottom: "DummyData1"
  top: "Convolution1"
  convolution_param {
    num_output: 64
    pad: 2
    kernel_size: 5
    group: 1
    stride: 1
  }
}
layer {
  name: "ReLU1"
  type: "ReLU"
  bottom: "Convolution1"
  top: "Convolution1"
}
layer {
  name: "Pooling1"
  type: "Pooling"
  bottom: "Convolution1"
  top: "Pooling1"
  pooling_param {
    pool: MAX
    kernel_size: 2
    stride: 2
  }
}
layer {
  name: "Convolution2"
  type: "Convolution"
  bottom: "Pooling1"
  top: "Convolution2"
  convolution_param {
    num_output: 128
    pad: 2
    kernel_size: 5
    group: 1
    stride: 1
  }
}
layer {
  name: "ReLU2"
  type: "ReLU"
  bottom: "Convolution2"
  top: "Convolution2"
}
layer {
  name: "Pooling2"
  type: "Pooling"
  bottom: "Convolution2"
  top: "Pooling2"
  

In [111]:
768 / 64.

12.0

In [99]:
1152 / 64

18